# Welcome to FOSSIL!
Overview

This tool uses neural networks alongside the powerful CEGIS architecture to automatically synthesise sound Lyapunov functions of Barrier Certificates for any N-dimensional system.


This notebook provides access to the following settings:

1. ### [Verifier Type](#verifier-type)
1. ### [Neural Network Structure](#nn-structure) 
    i)  Activation functions \
    ii) Layer structure (number of neurons, number of layers)
1. ### [Factorisation](#factorisation)
1. ### [Last Layer of Ones](#ll1)
1. ### [Barrier Function Sythesis Settings](#barrier-settings)
1. ### [Sympy Settings](#sympy-settings)
1. ### [Further CEGIS Settings](#further-settings) \
    i)   Learning Rate \
    ii)  Batch size \
    iii) Maximum number of CEGIS iterations \
    iv)  Rounding \
1. ### [Primer Settings](#primer-settings)
    
    
### [Running the tool with custom settings](#running)

If you would like to use a more simple interface to the tool with just defualt settings, please use the [playground](FOSSIL-playground.ipynb).



In [1]:
# % Imports
import sys
sys.path.append('..')
from experiments.playground_utils import *

<a id='verifier-type'></a>
## 1. Verifier Type

The tool has inbuilt support for both Z3 and Dreal4, and either can be selected as the backend to the verification. Please note that certain functionality cannot be used with Z3 as the verifier, though this is the default verifier.

**To specify the verifier, change the following variable to either VerifierType.Z3 or VerifierType.DREAL**


In [2]:
verifier_type = VerifierType.Z3

<a id='nn-structure'></a>
## 2. Neural Network Structure

The following activation functions are available:

* ### Linear or identity:
$$\sigma(x) = x $$
___

* ### Square:
$$\sigma(x) = x^2 $$
___

* ### Mixed $n^{th}$ order Polynomial:
$$ \sigma(x) = \sum_{i=1}^{n}x_i ^i, $$ where $x_i$ represents the i^th neurons in the layer, giving a mixed layer of activations. For more details on this please see [mixed_activation_functions]
___

* ### RELU:
$$ \sigma(x) = \max(0,x) $$
___

* ### RELU-Square:
$$\sigma(x) = x_1^2 + \max(0, x_2), $$

where x_1 denotes the the first half of the neurons in the layer and x_2 denotes the second half. 
___

* ### REQU:
$$ \sigma(x) = x \cdot \max(0,x)$$
___

* ### Sigmoid (requires Dreal):
$$ \sigma(x) = \frac{\text{e}^{x}}{\text{e}^{x} +1}$$
___

* ### Tanh (requires Dreal):
$$\sigma(x) = \tanh(x) $$


### Neuron Structure

The tool supports any number of layers each with any number of neurons and activation function. However, it should be noted that larger networks will have longer verification times. 

The desired activation functions for each should be inlcuded as a list with each element representing the activation function for that layer. Similarly, the number of neurons in each layer should be inlcuded as a list.
For example, for a network structure with two hidden layers, one with 50 and one with 20 neurons, the first of which has mixed second-order polynomial (LIN_SQUARE)  activations and the second with relu (RELU) activations.

activations = \[ ActivationType.LIN_SQUARE, ActivationType.RELU \] 

neurons     = \[ 50, 20 \]

The default parameters are given in the cell below and can be changed freely.

In [3]:
activations = [ActivationType.SQUARE]
neurons = [10]

<a id='factorisation'></a>
## 3. Factorisation

In order to help ensure that $V(x^*) = 0$ when synthesising Lyapunov r functions, the learner can be augmented such that the template form becomes $$ V(x) = (x-x^*) \cdot \text{n}(x)  $$ where $n(x)$ is the original neural network. Alternatively, a quadratic term can be used as $$ V(x) = (x-x^*)^2 \cdot \text{n}(x). $$ For more details on the factorisation please see [factorisation](../tex/factorisation.tex).

**To use linear factors, set the following variable to 'lf'.** \
**To use quadratic factors, set the following variable to 'qf'.** \
**To not use any factors (the defualt setting), set the following variable to None**

In [4]:
factors = None

## 4. Last Layer of Ones (Lyapunov Only)
llo(*boolean, defualt True*): This constrains the last layer of the neural network to be all ones. This can improve synthesise by helping ensure the positive definiteness (V(x) > 0) condition holds for the learner and verifier.

**To constrain the last layer to be all ones, set the following variable to True.**

<a id='ll1'></a>

In [5]:
llo = True

<a id='barrier-settings'></a>
## 5. Symettric Belt (Barrier Only)
*symmetric_belt (boolean, default=False)*: defines whether the belt for the derivative barrier condition is symmetric around zero (if so, we consider $|B(x)| \leq 0.5$) or not (if so, the we consider $B(x) \geq -0.1$). 

**To constrain the belt to be symmetric, set the following variable to True.**

In [6]:
symmetric_belt = False

## 6. Sympy Settings
<a id='sympy-settings'></a>

These settings determine the usage of sympy within the CEGIS object.

sp_handle (*boolean, default True*): determines whether expressions are handled using the python symbolic library *sympy*.

sp_simplify (*boolean, default True*): determines whether expressions are simplified using the *sympy* (potentially costly operation).

**To disable sympy handling and simplification, set the following variables to False.** 


In [7]:
sp_handle = True

sp_simplify = True

## 7. Further CEGIS Settings
<a id='further-settings'></a>
* Learning Rate (*positive float, default = 0.1*): sets the learning rate of the neural network.


* Batch Size (*int, defualt = 500*): defines the number of data points initially generated.


* Max Iterations (*int, default = 10*): sets the maximum number of CEGIS loops before termination



In [8]:
learning_rate = 0.1

batch_size = 500

max_iterations = 10

rounding = 3

<a id='primer-settings'></a>
### 8. Primer Settings

These settings affect how CEGIS is called and interacted with.

*seedbomb (boolean, default=False)*: if *seedbomb* is True, then a CEGIS object is repeatedly instantiated on a short timeout until the procedure is successful.

*interactive_domain (boolean, defualt=False)* : determines whether the user can adjust the domain size if CEGIS fails. Cannot be used in conjunction with *seedbomb*.

*positive_domain (boolean, defualt=True)*: Lyapunov only. If True, then the verification domain for Lyapunov conditions is constrained to the positive orthant.

In [15]:
seedbomb = False
positive_domain = True
domain_mode = 'auto'

## Running with custom settings

Once you have set custom settings using the cells above, use the following cell to instantiate a dynamical system and to synthesise either a Lyapunov or Barrier Certificate.
<a id='running'></a>

In [13]:
N_Dimensions = 2

x0, x1 = initialise_states(N_Dimensions)

dynamics = [
    -x0 + x0 * x1,
    -x1
]

mode = 'l'  # Change to 'b' to synthesise Barrier Certificate

parameters = {'VERIFIER':verifier_type, 'ACTIVATIONS': activations, 'NEURONS':neurons, 'FACTORS':factors, 
              'LLO':llo, 'LEARNING_RATE':learning_rate, 'BATCH_SIZE':batch_size, 'CEGIS_MAX_ITERS':max_iterations,
             'SP_HANDLE': sp_handle, 'SP_SIMPLIFY':sp_simplify,}

In [14]:
synthesise(dynamics, mode, SEEDBOMB=seedbomb, POSITIVE_DOMAIN=positive_domain, DOMAIN_MODE=domain_mode,
           CEGIS_PARAMETERS=parameters)

Single Equilibrium point found: 
 [(0, 0)]
Single Equilibrium point found: 
 [(0, 0)]
  learner   0
0 - loss: 33.802921295166016 - acc: 69.0 %
  regulariser   0
  verifier   0
Counterexample Found: [x1 = 3/2, x0 = 79/8]
V(ctx) =  15400235397555439910666729578303037079/640000000000000000000000000000000000
Vdot(ctx) =  109178249478152431487993683395259119/640000000000000000000000000000000000
  trajectoriser   0
  learner   1
0 - loss: 0.00886671245098114 - acc: 95.80152671755725 %
  regulariser   1
  verifier   1
Counterexample Found: [x1 = 25/16, x0 = 79/8]
V(ctx) =  34755430180381895120713762519103366191/640000000000000000000000000000000000
Vdot(ctx) =  412952633412137125205139435161780961/1280000000000000000000000000000000000
  trajectoriser   1
  learner   2
0 - loss: 32184.734375 - acc: 95.8029197080292 %
100 - loss: 37.25857925415039 - acc: 77.18978102189782 %
200 - loss: 28.834341049194336 - acc: 77.18978102189782 %
300 - loss: 21.89669418334961 - acc: 77.55474452554745 %
400 - lo